In [1]:
!nvidia-smi

Sat Aug 15 14:59:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!cp '/content/drive/My Drive/cub/CUB_200_2011.tgz' '/content/'
!tar zxf  CUB_200_2011.tgz
!rm CUB_200_2011.tgz

In [22]:
!cp -r '/content/drive/My Drive/valid/' '/content'

In [ ]:
pip install scipy==1.1.0

In [23]:
import os
__name__ = '__main__'
if __name__ == '__main__':
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    paths = ["",""]
    paths[0] = '/content/CUB_200_2011/images/'
    paths[1] = '/content/valid/single/'
    print(paths)
    fid_value = calculate_fid_given_paths(paths, 50 ,'0',2048)
    print('FID: ', fid_value)

['/content/CUB_200_2011/images/', '/content/valid/single/']
11788
----------------------------------------------------------------
Propagating batch 235/235 done
mu:  [0.15935878 0.33772494 0.18450351 ... 0.22667881 0.36900787 0.1993268 ]
sigma:  [[ 0.01664562 -0.00327295  0.00369965 ...  0.00113071  0.00083431
   0.00539438]
 [-0.00327295  0.05203868 -0.00190443 ... -0.00122505  0.00914806
   0.00081765]
 [ 0.00369965 -0.00190443  0.0197717  ...  0.00356483  0.00350863
   0.00198308]
 ...
 [ 0.00113071 -0.00122505  0.00356483 ...  0.05193557  0.01538079
   0.01045969]
 [ 0.00083431  0.00914806  0.00350863 ...  0.01538079  0.06135809
   0.00956308]
 [ 0.00539438  0.00081765  0.00198308 ...  0.01045969  0.00956308
   0.03452198]]
2912
----------------------------------------------------------------
Propagating batch 58/58 done
mu:  [0.19243671 0.37091766 0.21827795 ... 0.13723144 0.36766818 0.195555  ]
sigma:  [[ 0.02205793 -0.00476923  0.00450087 ...  0.00113227  0.00148771
   0.007572

In [17]:
#!/usr/bin/env python3
"""Calculates the Frechet Inception Distance (FID) to evalulate GANs
The FID metric calculates the distance between two distributions of images.
Typically, we have summary statistics (mean & covariance matrix) of one
of these distributions, while the 2nd distribution is given by a GAN.
When run as a stand-alone program, it compares the distribution of
images that are stored as PNG/JPEG at a specified location with a
distribution given by summary statistics (in pickle format).
The FID is calculated by assuming that X_1 and X_2 are the activations of
the pool_3 layer of the inception net for generated samples and real world
samples respectivly.
See --help to see further details.
Code apapted from https://github.com/bioinf-jku/TTUR to use PyTorch instead
of Tensorflow
Copyright 2018 Institute of Bioinformatics, JKU Linz
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""
import os
import pathlib
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

import torch
import numpy as np
from scipy.misc import imread
from scipy import linalg
from torch.autograd import Variable
from torch.nn.functional import adaptive_avg_pool2d
import torchvision.transforms as transforms
import torch.utils.data
from PIL import Image
from torch.utils import data
import img_data as img_data
from inception import InceptionV3

# parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
# #parser.add_argument('path', type=str, nargs=2,
# #                    help=('Path to the generated images or '
# #                          'to .npz statistic files'))
# parser.add_argument('--batch-size', type=int, default=64,
#                     help='Batch size to use')
# parser.add_argument('--dims', type=int, default=2048,
#                     choices=list(InceptionV3.BLOCK_INDEX_BY_DIM),
#                     help=('Dimensionality of Inception features to use. '
#                           'By default, uses pool3 features'))
# parser.add_argument('-c', '--gpu', default='', type=str,
#                     help='GPU to use (leave blank for CPU only)')
# parser.add_argument('--path1', type=str, default=64)
# parser.add_argument('--path2', type=str, default=64)

def get_activations(images, model, batch_size=64, dims=2048, cuda=False, verbose=True):
    """Calculates the activations of the pool_3 layer for all images.
    Params:
    -- images      : Numpy array of dimension (n_images, 3, hi, wi). The values
                     must lie between 0 and 1.
    -- model       : Instance of inception model
    -- batch_size  : the images numpy array is split into batches with
                     batch size batch_size. A reasonable batch size depends
                     on the hardware.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the number
                     of calculated batches is reported.
    Returns:
    -- A numpy array of dimension (num images, dims) that contains the
       activations of the given tensor when feeding inception with the
       query tensor.
    """
    model.eval()

    #d0 = images.shape[0]

    d0 = images.__len__() * batch_size
    if batch_size > d0:
        print(('Warning: batch size is bigger than the data size. '
               'Setting batch size to data size'))
        batch_size = d0

    n_batches = d0 // batch_size
    n_used_imgs = n_batches * batch_size

    pred_arr = np.empty((n_used_imgs, dims))
    #for i in range(n_batches):
    for i, batch in enumerate(images):
        #batch = batch[0]
        if verbose:
            print('\rPropagating batch %d/%d' % (i + 1, n_batches), end='', flush=True)
        #import ipdb
        #ipdb.set_trace()
        start = i * batch_size
        end = start + batch_size

        #batch = torch.from_numpy(images[start:end]).type(torch.FloatTensor)
        #batch = Variable(batch, volatile=True)

        if cuda:
            batch = batch.cuda()
        #print('The Model: ')
        #print(model)
        #print('The Batch: ')
        #print(batch.size())torch.Size([50, 3, 299, 299]) 
        pred = model(batch)[0]
        #print('The pred: ')
        #print(pred.size())torch.Size([50, 2048, 1, 1])

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
        if pred.shape[2] != 1 or pred.shape[3] != 1:
            pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

        pred_arr[start:end] = pred.cpu().data.numpy().reshape(batch_size, -1)
        #print('pred_arr')
        #print(pred_arr.size) 24064000

    if verbose:
        print(' done')

    return pred_arr


def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an 
               representive data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an 
               representive data set.
    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)


def calculate_activation_statistics(images, model, batch_size=64,
                                    dims=2048, cuda=False, verbose=True):
    """Calculation of the statistics used by the FID.
    Params:
    -- images      : Numpy array of dimension (n_images, 3, hi, wi). The values
                     must lie between 0 and 1.
    -- model       : Instance of inception model
    -- batch_size  : The images numpy array is split into batches with
                     batch size batch_size. A reasonable batch size
                     depends on the hardware.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the
                     number of calculated batches is reported.
    Returns:
    -- mu    : The mean over samples of the activations of the pool_3 layer of
               the inception model.
    -- sigma : The covariance matrix of the activations of the pool_3 layer of
               the inception model.
    """
    print('----------------------------------------------------------------')
    act = get_activations(images, model, batch_size, dims, cuda, verbose= True)
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    print('mu: ', mu )
    print('sigma: ', sigma )
    return mu, sigma

def _compute_statistics_of_path(path, model, batch_size, dims, cuda):
    if path.endswith('.npz'):
        f = np.load(path)
        m, s = f['mu'][:], f['sigma'][:]
        f.close()

    else:
        dataset = img_data.Dataset(path, transforms.Compose([
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
        ]))
        print(dataset.__len__())
        dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=8)
        m, s = calculate_activation_statistics(dataloader, model, batch_size, dims, cuda)
    return m, s

def calculate_fid_given_paths(paths, batch_size, cuda, dims):
    """Calculates the FID of two paths"""
    for p in paths:
        if not os.path.exists(p):
            raise RuntimeError('Invalid path: %s' % p)

    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]

    model = InceptionV3([block_idx])
    #model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)

    if cuda:
        model.cuda()

    m1, s1 = _compute_statistics_of_path(paths[0], model, batch_size, dims, cuda)
    m2, s2 = _compute_statistics_of_path(paths[1], model, batch_size, dims, cuda)
    fid_value = calculate_frechet_distance(m1, s1, m2, s2)
    return fid_value